In diesem Notebook sind die Ergebnisse des BigData Projektes der Gruppe 5 zusammengefasst. 
Als Analyseobjekt dient die Wettersimulation. Hierfür wird auf der Basis von Wetterdaten des Deutschen Wetterdienstes (DWD) als Beispiel die Bauernregel "Ist der Januar hell und weiß, wird der Sommer sicher heiß" auf seine Korrektheit überprüft. 

In [1]:
#Set up von Spark
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName("BigData").getOrCreate()
sc = spark.sparkContext
spark

23/02/03 02:39:21 WARN Utils: Your hostname, mario-hpprobook450g5 resolves to a loopback address: 127.0.1.1; using 141.47.138.65 instead (on interface wlp3s0)
23/02/03 02:39:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/03 02:39:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#Speicherorte der Daten festlegen 
#Daniel
#folder_Temperature = '/Users/danielwentsch/Desktop/BigData/Data/02_airtemperature/'
#folder_Solar = '/Users/danielwentsch/Desktop/BigData/Data/04_solar/'
#Mario
folder_Temperature = '/home/mario/dwd_data/temperatur_only_10_stations/'
folder_Solar = '/home/mario/dwd_data/solar_only_10_stations/'

In [3]:
#Einlesen der Daten in einen DataFrame

#Lufttemperatur
#Daniel
df_Temperatur = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Temperature+'*.txt')
#Sonnenscheindauer
#Daniel
df_Solar = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Solar+'*.txt')

In [4]:
#df_Temperatur.count()

In [5]:
#df_Solar.count()

Nächste Schritte: 
1. Datenstruktur verstehen 
2. Daten bearbeiten und in einen finalen DF schreiben 
-> Zusammenfassen der Daten auf eine Station und einen TimeStamp

In [6]:
df_Temperatur.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- PP_10: string (nullable = true)
 |-- TT_10: string (nullable = true)
 |-- TM5_10: string (nullable = true)
 |-- RF_10: string (nullable = true)
 |-- TD_10: string (nullable = true)
 |-- eor: string (nullable = true)



Aus diesem Datensatz benötigen wir die Spalten StationsID und MESS_Datum (Zeitstempel) TT_10 (Luftemperatur in 2m Höhe) und TM5_10 (Luftemperatur in 2cm Höhe) und PP_10(Luftdruck)
Ändern der Datentypen von TT_10, TM5_10 und PP_10 in float, MESS_Datum in Timestamp ändern

In [7]:
#Bearbeiten der Struktur in Gewünschtes Format 
#Datentypen bearbeiten 
df_Temperatur = df_Temperatur\
    .withColumn('STATIONS_ID',df_Temperatur.STATIONS_ID.cast(IntegerType()))\
    .withColumn('TM5_10',df_Temperatur.TM5_10.cast(FloatType()))\
    .withColumn('PP_10',df_Temperatur.PP_10.cast(FloatType()))\
    .withColumn('TT_10',df_Temperatur.TT_10.cast(FloatType()))\
    .withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))
#Daten droppen
df_Temperatur = df_Temperatur.drop('  QN','RF_10','TD_10','eor')
#Spalten umbennen
df_Temperatur = df_Temperatur.withColumnRenamed('TT_10','Temperatur_2m').withColumnRenamed('TM5_10','Temperatur_5cm').withColumnRenamed('PP_10','Luftdruck')
df_Temperatur = df_Temperatur.filter((df_Temperatur.Luftdruck != -999.0))
df_Temperatur = df_Temperatur.filter((df_Temperatur.Temperatur_2m != -999.0))
df_Temperatur = df_Temperatur.filter((df_Temperatur.Temperatur_5cm != -999.0))
df_Temperatur.show(3,False)

+-----------+-------------------+---------+-------------+--------------+
|STATIONS_ID|MESS_DATUM         |Luftdruck|Temperatur_2m|Temperatur_5cm|
+-----------+-------------------+---------+-------------+--------------+
|1766       |2010-01-01 00:00:00|989.7    |-1.3         |-1.8          |
|1766       |2010-01-01 00:10:00|989.7    |-1.3         |-1.8          |
|1766       |2010-01-01 00:20:00|989.8    |-1.3         |-1.8          |
+-----------+-------------------+---------+-------------+--------------+
only showing top 3 rows



In [8]:
df_Temperatur.count()

11252290

In [9]:
df_Solar.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- DS_10: string (nullable = true)
 |-- GS_10: string (nullable = true)
 |-- SD_10: string (nullable = true)
 |-- LS_10: string (nullable = true)
 |-- eor: string (nullable = true)



Aus diesem Datensatz benötigen wir die Spalten StationsID und MESS_Datum (Zeitstempel), SD_10 (10 min-Summe der Sonnenscheindauer)
Ändern der Datentypen von SD_10 in float

In [10]:
#Datentypen bearbeiten
df_Solar = df_Solar.withColumn('SD_10',df_Solar.SD_10.cast(FloatType())).withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))
#Daten droppen 
df_Solar = df_Solar.drop('  QN','DS_10','GS_10','LS_10','eor')
#Spalte numbennen
df_Solar = df_Solar.withColumnRenamed('SD_10','Sonnenscheindauer')
df_Solar = df_Solar.filter((df_Solar.Sonnenscheindauer != -999.0))
df_Solar.show(3,False)

+-----------+-------------------+-----------------+
|STATIONS_ID|MESS_DATUM         |Sonnenscheindauer|
+-----------+-------------------+-----------------+
|        232|2010-01-01 00:00:00|0.0              |
|        232|2010-01-01 00:10:00|0.0              |
|        232|2010-01-01 00:20:00|0.0              |
+-----------+-------------------+-----------------+
only showing top 3 rows



In [11]:
#df_Solar.count()

Nachdem die beiden Datensätze in die passende Struktur gebracht wurden, können Sie mit einem Join verbunden werden. 
Welchen Join? 
Da für die Auswertung des Spruches, für jede Station und jeden TimeStamp Daten von Solar und Temperature benötigt werden, müssen diese auch von beiden vorhanden sein. 
Daher wird der Inner Join gewählt. 

In [12]:
#Join der DF
df_Final = df_Temperatur.join(df_Solar, (df_Temperatur.STATIONS_ID == df_Solar.STATIONS_ID) & (df_Temperatur.MESS_DATUM == df_Solar.MESS_DATUM),"inner").select(df_Temperatur['*'],df_Solar.Sonnenscheindauer)

In [13]:
df_Final.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- Luftdruck: float (nullable = true)
 |-- Temperatur_2m: float (nullable = true)
 |-- Temperatur_5cm: float (nullable = true)
 |-- Sonnenscheindauer: float (nullable = true)



In [14]:
#df_Final.show(3,False)

In [15]:
#df_Final.count()

Auf der Basis dieser Datentabelle kann nun die Auswertung zur Bauernregel erfolgen. 
Diese Tabelle kann natürlich noch um weitere Wetterdaten erweitert werden, damit andere Bauernregeln oder andere Wettersimulationen durchgeführt werden können.

Nun erfolgt die Auswertung des Spruches: "Ist der Januar hell und weiß, wird der Sommer sicher heiß"
1. DF anlegen, welcher alle Kombinationen von Station ID und Jahre besitzt
2. Daten für Januar und Sommer erstellen
3. Auswertung

In [16]:
#DF aus dem df_Final erzeugen welcher alle Kombinationen der StationsIDs, Monate und Jahre enthält
#Neue Spalte Jahr hinzufügen
df_Final  = df_Final.withColumn("Jahr", year("MESS_DATUM"))
df_Final  = df_Final.withColumn("Monat", month("MESS_DATUM"))
df_Final.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- Luftdruck: float (nullable = true)
 |-- Temperatur_2m: float (nullable = true)
 |-- Temperatur_5cm: float (nullable = true)
 |-- Sonnenscheindauer: float (nullable = true)
 |-- Jahr: integer (nullable = true)
 |-- Monat: integer (nullable = true)



In [17]:
df_Kombinationen =  df_Final.withColumn("ID",concat_ws('','Jahr','STATIONS_ID'))
df_Kombinationen = df_Kombinationen.select("ID",'STATIONS_ID','Jahr')
df_Kombinationen = df_Kombinationen.dropDuplicates(["ID"])
df_Kombinationen.printSchema()

root
 |-- ID: string (nullable = false)
 |-- STATIONS_ID: integer (nullable = true)
 |-- Jahr: integer (nullable = true)



In [18]:
df_Kombinationen.show(3,False)

23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:41:16 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-----------+----+
|ID      |STATIONS_ID|Jahr|
+--------+-----------+----+
|20151766|1766       |2015|
|20161766|1766       |2016|
|20183166|3166       |2018|
+--------+-----------+----+
only showing top 3 rows



Pro Jahr sind nur die Temperaturen von 21. Juni bis 23. September und die Sonnenscheindauer im Januar wichtig 

In [19]:
#Zu DF df_Kombinationen eine neue Spalte hinzufügen, welche die Summe an Sonnenstunden pro StationsID und Jahr enthält 
df_Final_Solar_Grouped = df_Final.filter((df_Final.Monat == 12)).groupBy('STATIONS_ID','Jahr','Monat').mean('Sonnenscheindauer')
df_Final_Solar_Grouped = df_Final_Solar_Grouped.withColumnRenamed('avg(Sonnenscheindauer)','Sonnenscheindauer')
df_Kombinationen = df_Kombinationen.join(df_Final_Solar_Grouped,(df_Kombinationen.STATIONS_ID == df_Final_Solar_Grouped.STATIONS_ID)&(df_Kombinationen.Jahr == df_Final_Solar_Grouped.Jahr )).select(df_Kombinationen['*'],df_Final_Solar_Grouped.Sonnenscheindauer)

In [20]:
df_Final_Temperatur_Grouped = df_Final.filter((df_Final.Monat == 7)|(df_Final.Monat == 8)).groupBy('STATIONS_ID','Jahr').mean('Temperatur_2m')
df_Final_Temperatur_Grouped = df_Final_Temperatur_Grouped.withColumnRenamed('avg(Temperatur_2m)','Temperatur_2m')
df_Kombinationen = df_Kombinationen.join(df_Final_Temperatur_Grouped,(df_Kombinationen.STATIONS_ID == df_Final_Temperatur_Grouped.STATIONS_ID)&(df_Kombinationen.Jahr == df_Final_Temperatur_Grouped.Jahr )).select(df_Kombinationen['*'],df_Final_Temperatur_Grouped.Temperatur_2m)
df_Kombinationen = df_Kombinationen.sort('Jahr')
df_Kombinationen = df_Kombinationen.sort('STATIONS_ID')

In [21]:
df_Kombinationen.printSchema()

root
 |-- ID: string (nullable = false)
 |-- STATIONS_ID: integer (nullable = true)
 |-- Jahr: integer (nullable = true)
 |-- Sonnenscheindauer: double (nullable = true)
 |-- Temperatur_2m: double (nullable = true)



In [22]:
df_Kombinationen.show(3,False)

23/02/03 02:45:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:45:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:45:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:45:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:45:23 WARN RowBasedKeyValueBatch: Calling spill() on

+-------+-----------+----+--------------------+------------------+
|ID     |STATIONS_ID|Jahr|Sonnenscheindauer   |Temperatur_2m     |
+-------+-----------+----+--------------------+------------------+
|2015232|232        |2015|0.025358198403370797|20.47743055197714 |
|2012232|232        |2012|0.013385596887166921|18.321281366931494|
|2011232|232        |2011|0.007090725703999453|17.31738350874207 |
+-------+-----------+----+--------------------+------------------+
only showing top 3 rows



In [23]:
#Korrelation zwischen Sonnenschein und Durschsnittstemperatur berechnen
corr_Spruch = df_Kombinationen.stat.corr('Sonnenscheindauer','Temperatur_2m')

23/02/03 02:48:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:49:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:49:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:49:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:49:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/03 02:49:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/03 02:49:08 WARN RowBasedKeyValueBatch: Calling spill() on

In [24]:
print(corr_Spruch)

0.07188381758199253
